<a href="https://colab.research.google.com/github/Brandon12231/Business-Analytics-Toolbox/blob/master/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this homework fit and select a classifier to predict credit card default using `default_of_credit_card_clients` dataset from BlackBoard course site. The data description is available at : https://www.kaggle.com/datasets/jishnukoliyadan/taiwan-default-credit-card-clients

1. Explore (5+5+10=20 points)
  1. load the dataset. Use only the columns "LIMIT_BAL", "SEX", "EDUCATION", "MARRIAGE", and "AGE" among predictors. The target is "default payment next month". 
  1. identify the categorical features (with brief explanation), and 
  1. produce the pairwise scatter plot only for the numeric variables. 
1. Prepare a pipeline to (30 points)
  1. standardize the numeric attributes
  1. expand the categorical attributes to columns of 0/1 variables
  1. fit a `RandomForestClassifier` classifier
1. Search over the `max_depth` and `min_samples_leaf` parameters to find the best model per **balanced accuracy** metric. Use at least three different search strategies and discuss any differences you see in the results. (20 points)
1. Let's assume that the cost of missing a default (i.e., predicting non-default for a customer who ended up defaulting) is 10 times the cost of flagging a non-defaulter as defaulter. Let's further assume that the cost of correct predictions are 0. Use any one of the search strategies considered in the previous question to find the `RandomForestClassifier` that minimizes the cost. (20 points)
1. Collaboration statement: Who did you discuss while answering this homework (whether to get or to provide help)? What questions/topics did you discuss? (10 points)

Note: No penalty for either side. While getting help in figuring out how to solve is OK, all answers should be produced by you. 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')
data_folder = '/content/drive/MyDrive/data/810/default_of_credit_card_clients.csv/'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_folder = '/content/drive/MyDrive/data/810/default_of_credit_card_clients.csv/'

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/data/810/default_of_credit_card_clients.csv')
data.info()
data.head(3)

In [ ]:
data=data[["LIMIT_BAL","SEX","EDUCATION","MARRIAGE","AGE","default payment next month"]]
data

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size = .25, random_state=0)

train_data.shape, test_data.shape

In [ ]:
train_data.describe()

In [ ]:
train_data["SEX"].value_counts()

In [ ]:
train_data["EDUCATION"].value_counts()

In [ ]:
train_data["MARRIAGE"].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
data1=data[["LIMIT_BAL","AGE"]]
sns.pairplot(data1, hue=None, hue_order=None, palette=None, vars=None, x_vars=None, y_vars=None, kind='scatter', diag_kind='hist', markers=None, size=2.5, aspect=1, dropna=True, plot_kws=None, diag_kws=None, grid_kws=None)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn import set_config

set_config(display='diagram') # shows the pipeline structure graphically

num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

In [ ]:
from sklearn.preprocessing import OneHotEncoder
cat_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("cat_encoder", OneHotEncoder(sparse=False, drop="first"))
    ])

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

num_attribs = ["LIMIT_BAL","AGE"]
cat_attribs = ["SEX","EDUCATION","MARRIAGE"]

# The following step patches SimpleImputer to produce feature names
SimpleImputer.get_feature_names_out = StandardScaler.get_feature_names_out

preprocess_pipeline = ColumnTransformer([ # handle each type of column with appropriate pipeline
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])

full_pipeline = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('svm', SVC()),
])

In [ ]:
df_X = train_data.drop("default payment next month", axis=1)
df_y = train_data["default payment next month"]
print(df_X.head())  # to check that 'y' isn't included

X_train = preprocess_pipeline.fit_transform(df_X)
y_train = df_y.values
preprocess_pipeline.get_feature_names_out() # check the column names produced by the pipeline

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
forest_clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score

X_test = preprocess_pipeline.transform(test_data)
y_test = test_data["default payment next month"]
y_pred = forest_clf.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
balanced_accuracy_score(y_test, y_pred).round(4)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
import matplotlib
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_predictions(y_test, y_pred);

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import RocCurveDisplay
RocCurveDisplay.from_estimator(forest_clf, X_test, y_test);

In [ ]:
from sklearn.model_selection import cross_val_score

forest_scores = cross_val_score(forest_clf, X_train, y_train, cv=10, scoring='balanced_accuracy')
forest_scores.mean().round(3)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

lr_pipe = make_pipeline(preprocess_pipeline, LogisticRegression())
# we have to use data frames here because the preprocessing step expects them.
lr_scores = cross_val_score(lr_pipe, df_X, df_y, cv=10, scoring='balanced_accuracy')
lr_scores.mean().round(3)

In [ ]:
#GRID
from sklearn.model_selection import GridSearchCV

param_grid = [ 
    {'svm__C': np.logspace(-3, 3, 4), 
     'svm__gamma': np.logspace(-2, 2, 5)
     }, 
    ]
# Check what's in this parameter grid
print('The parameter grid : ')
print(param_grid)

grid_search = GridSearchCV(full_pipeline, param_grid, cv=3, 
                                 scoring='balanced_accuracy')
grid_search.fit(df_X, df_y)
print('\n\nThe best parameters are ', grid_search.best_params_)

grid_cv_res = pd.DataFrame(grid_search.cv_results_) # convert to DF for convenience
grid_cv_res.sort_values(by='mean_test_score', ascending=False, inplace=True)  # sort the data frame
# select only the columns that start with 'param_' and the column 'mean_test_score'
grid_cv_res.filter(regex = '(^param_|mean_test_score)', axis=1).head()

The parameter grid : 
[{'svm__C': array([1.e-03, 1.e-01, 1.e+01, 1.e+03]), 'svm__gamma': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])}]
